In [ ]:
import pandas as pd

data_2021_2023_rakuten = pd.read_csv("../csv/Rakuten/sales_data_2021-2023_rakuten.csv", encoding="utf-8-sig")
data_2021_2023_yahoo = pd.read_csv("../csv/Yahoo/sales_data_2021-2023-06-13_yahoo.csv", encoding="utf-8-sig")
data_2023_2024 = pd.read_csv("../csv/r_y_sale_data.csv", encoding="utf-8-sig")
data_2024_2025_rakuten = pd.read_csv("../dataset/final_20240422-20250412.csv", encoding="utf-8-sig")
data_2024_2025_yahoo = pd.read_csv("../dataset/final_yahoo_20240422-2025-0412.csv", encoding="utf-8-sig")

columns_yahoo = ["OrderDay", "Title", "TotalPrice（商品合計金額）","TotalMallCouponDiscount（クーポン値引金額）","ShipCharge（送料）"]
columns_rakuten = ["OrderDay", "Title", "TotalPrice（商品合計金額）","TotalMallCouponDiscount（クーポン値引金額）","ShipCharge（送料）"]
columns_2024_2025_rakuten = ["注文日","商品名","合計金額","送料合計","クーポン利用総額"]
columns_2024_2025_yahoo = ["OrderTime","Title","UnitPrice","TotalMallCouponDiscount","ShipCharge"]

new_columns = ["OrderTime","Title", "TotalPrice","TotalMallCouponDiscount",	"ShipCharge"]


,注文日,商品名,合計金額,送料合計,クーポン利用総額
0,2024-04-26,美品【中古】ピエールカルダン pierre cardin お洒落ジャケット 表記Mサイズ(9...,25600,0,2000
1,2024-04-26,美品【中古】ポールスミス Poul Smith テーラードジャケット ジャケット 表記 38...,25600,0,2000
2,2024-04-26,新品同様【中古】ホコモモラ Jocomomola 緑 涼感 素敵ブラウス トップス 42号/...,10800,0,1000
3,2024-04-27,美品【中古】組曲 KUMIKYOKU ワンピース 異素材ワンピース 大きいサイズ 表記 5号...,21900,0,0
4,2024-05-02,美品【中古】ミラオーウェン mila owen タイトスカート ニットスカート スカート レ...,16300,0,0
...,...,...,...,...,...
400,2025-03-27,美品【中古】ジブリーヌ Zibeline 東京イギン フォーマルジャケット ノーカラージャケ...,33500,0,0
401,2025-03-29,新品同様【中古】スキャパ SCAPA フレアスカート 表記 38号 9号 M 相当 古着 紺...,8900,0,0
402,2025-04-03,美品【中古】レオナール LEONARD デニムスカート 膝丈スカート スカート レディース ...,28780,0,0
403,2025-04-08,新品同様【中古】エプリス EPRISE NOIR3 東京ブラウス フォーマルブラウス ブラウ...,20000,0,2000


In [26]:
import pandas as pd
import re

# --- データの結合 ---

# 必要な列だけを抽出（列名は事前に定義済みであると仮定）
all_data_2021_2023 = pd.concat(
    [data_2021_2023_rakuten[columns_rakuten], data_2021_2023_yahoo[columns_yahoo]],
    ignore_index=True
)

# Yahooと楽天の列名を統一（適宜調整）
all_data_2021_2023.rename(columns={
    "OrderDay": "OrderTime",
    "TotalPrice（商品合計金額）": "TotalPrice",
    "TotalMallCouponDiscount（クーポン値引金額）": "TotalMallCouponDiscount",
    "ShipCharge（送料）": "ShipCharge"
}, inplace=True)

# 2023-2024年のデータをさらに結合
all_data_2021_2023 = pd.concat(
    [all_data_2021_2023, data_2023_2024[columns]],
    ignore_index=True
)

# --- 日付形式修正 ---

# 正規表現で "YYYY/M/D H:MM" → "YYYY/M/D HH:MM" に変換（ゼロ埋め）
def fix_time_format(s):
    if pd.isna(s):
        return s
    # "2023/6/17 8:45" → "2023/6/17 08:45"
    return re.sub(r'(\d{4}/\d{1,2}/\d{1,2}) (\d:\d{2})', r'\1 0\2', str(s))

# 日付フォーマットを整形
all_data_2021_2023['OrderTime'] = all_data_2021_2023['OrderTime'].apply(fix_time_format)

# datetime型に変換
all_data_2021_2023['OrderTime'] = pd.to_datetime(all_data_2021_2023['OrderTime'], errors='coerce')

# 日付だけの列を作成
all_data_2021_2023['OrderDay'] = all_data_2021_2023['OrderTime'].dt.date

# OrderTimeで昇順ソート
all_data_2021_2023 = all_data_2021_2023.sort_values(by='OrderTime').reset_index(drop=True)

# 結果表示（確認用）
all_data_2021_2023


,OrderTime,Title,TotalPrice,TotalMallCouponDiscount,ShipCharge,OrderDay
0,2021-06-16,美品/東京ソワール D'REREAD ブラックフォーマルワンピース 大きいサイズ 表記15号...,27720,NaN,0,2021-06-16
1,2021-06-20,【中古】インディヴィ INDIVI 暖かウール混コート/ジャケット 表記38号(9号/M相当...,20000,0.0,0,2021-06-20
2,2021-06-22,【中古】23区 美形パンツスーツ 表記40号(11号/L相当) グレー 薄手 毛ウール混 シ...,18000,0.0,0,2021-06-22
3,2021-06-24,【中古】クリスチャンオジャール CHRISTIAN AUJARD 茶ブラウン 7分袖 美柄カ...,7200,0.0,0,2021-06-24
4,2021-07-02,【中古】斉藤都世子 toyoko saito ニットの女王 大人ニットワンピース 9号/38...,39500,500.0,0,2021-07-02
...,...,...,...,...,...,...
1897,NaT,A新品同様/ アナイ ANAYI 大人なニットカーディガン 表記38号(M相当) 黒/ブラッ...,6785,0.0,600,NaT
1898,NaT,美品*YUKIKO HANAI・ユキコハナイ*茶*花柄レース*素敵ブラウス*9,24900,0.0,0,NaT
1899,NaT,Ａ美品/ラピーヌブランシュ LAPINE BLANCHE 大人な美形ジャケット 表記42号(...,12800,0.0,0,NaT
1900,NaT,新品同様/チャコット Chacott 社交ダンススカート 表記 M 9号 相当 紺 ネイビー...,13750,0.0,0,NaT


In [30]:
import pandas as pd
import re

# CSV読み込み
data_2021_2023_rakuten = pd.read_csv("../csv/Rakuten/sales_data_2021-2023_rakuten.csv", encoding="utf-8-sig")
data_2021_2023_yahoo = pd.read_csv("../csv/Yahoo/sales_data_2021-2023-06-13_yahoo.csv", encoding="utf-8-sig")
data_2023_2024 = pd.read_csv("../csv/r_y_sale_data.csv", encoding="utf-8-sig")

# 使用する列の定義
columns_yahoo = ["OrderDay", "Title", "TotalPrice（商品合計金額）", "TotalMallCouponDiscount（クーポン値引金額）", "ShipCharge（送料）"]
columns_rakuten = ["OrderDay", "Title", "TotalPrice（商品合計金額）", "TotalMallCouponDiscount（クーポン値引金額）", "ShipCharge（送料）"]
columns_2023_2024 = ["OrderTime", "Title", "TotalPrice", "TotalMallCouponDiscount", "ShipCharge"]

# Yahoo・楽天を結合し列名統一
all_data_2021_2023 = pd.concat(
    [data_2021_2023_rakuten[columns_rakuten], data_2021_2023_yahoo[columns_yahoo]],
    ignore_index=True
).rename(columns={
    "OrderDay": "OrderTime",
    "TotalPrice（商品合計金額）": "TotalPrice",
    "TotalMallCouponDiscount（クーポン値引金額）": "TotalMallCouponDiscount",
    "ShipCharge（送料）": "ShipCharge"
})

# OrderTime をゼロ埋めする処理（時刻がある可能性に備えて）
def fix_time_format(s):
    if pd.isna(s):
        return s
    return re.sub(r'(\d{4}/\d{1,2}/\d{1,2}) (\d:\d{2})', r'\1 0\2', str(s))

all_data_2021_2023['OrderTime'] = all_data_2021_2023['OrderTime'].apply(fix_time_format)
all_data_2021_2023['OrderTime'] = pd.to_datetime(all_data_2021_2023['OrderTime'], errors='coerce')

# data_2023_2024のOrderTimeはすでにYYYY-MM-DD表記になっている前提
data_2023_2024['OrderTime'] = pd.to_datetime(data_2023_2024['OrderTime'], errors='coerce')

# データを統合
all_data = pd.concat(
    [all_data_2021_2023, data_2023_2024[columns_2023_2024]],
    ignore_index=True
)

# 日付列追加（OrderTimeから抽出）
all_data['OrderTime'] = all_data['OrderTime'].dt.strftime('%Y-%m-%d')

# 昇順にソート
all_data = all_data.sort_values(by='OrderTime').reset_index(drop=True)

all_data = all_data.fillna(0)  # NaNを0で埋める

# 結果を確認
all_data

# 必要なら保存
# all_data.to_csv("all_sales_data_sorted.csv", index=False, encoding="utf-8-sig")


,OrderTime,Title,TotalPrice,TotalMallCouponDiscount,ShipCharge
0,2021-06-16,美品/東京ソワール D'REREAD ブラックフォーマルワンピース 大きいサイズ 表記15号...,27720,0.0,0
1,2021-06-20,【中古】インディヴィ INDIVI 暖かウール混コート/ジャケット 表記38号(9号/M相当...,20000,0.0,0
2,2021-06-22,【中古】23区 美形パンツスーツ 表記40号(11号/L相当) グレー 薄手 毛ウール混 シ...,18000,0.0,0
3,2021-06-24,【中古】クリスチャンオジャール CHRISTIAN AUJARD 茶ブラウン 7分袖 美柄カ...,7200,0.0,0
4,2021-07-02,【中古】斉藤都世子 toyoko saito ニットの女王 大人ニットワンピース 9号/38...,39500,500.0,0
...,...,...,...,...,...
1897,2024-04-17,美品【中古】レリアン Leilian 美揺フレアスカート 表記9号(38号/M相当) 茶/ブ...,18300,0.0,0
1898,2024-04-17,H美品/コムサ COMME CA 美形ウール混ジャケット 表記11号(Lサイズ相当) 黒茶/...,5500,0.0,600
1899,2024-04-21,新品同様/ローザ ナラカミーチェ Rosa by NARACAMICIE 素敵チュニック ブ...,6800,100.0,600
1900,2024-04-21,美品/東京イギン TOKYO IGIN ブラックフォーマルワンピーススーツ 大きい 表記 2...,31900,0.0,0


In [34]:
import pandas as pd
import re

# --- ファイル読み込み ---
data_2021_2023_rakuten = pd.read_csv("../csv/Rakuten/sales_data_2021-2023_rakuten.csv", encoding="utf-8-sig")
data_2021_2023_yahoo = pd.read_csv("../csv/Yahoo/sales_data_2021-2023-06-13_yahoo.csv", encoding="utf-8-sig")
data_2023_2024 = pd.read_csv("../csv/r_y_sale_data.csv", encoding="utf-8-sig")
data_2024_2025_rakuten = pd.read_csv("../dataset/final_20240422-20250412.csv", encoding="utf-8-sig")
data_2024_2025_yahoo = pd.read_csv("../dataset/final_yahoo_20240422-2025-0412.csv", encoding="utf-8-sig")

# --- 使用カラム定義 ---
columns_rakuten = ["OrderDay", "Title", "TotalPrice（商品合計金額）", "TotalMallCouponDiscount（クーポン値引金額）", "ShipCharge（送料）"]
columns_yahoo = ["OrderDay", "Title", "TotalPrice（商品合計金額）", "TotalMallCouponDiscount（クーポン値引金額）", "ShipCharge（送料）"]
columns_2023_2024 = ["OrderTime", "Title", "TotalPrice", "TotalMallCouponDiscount", "ShipCharge"]
columns_2024_2025_rakuten = ["注文日", "商品名", "合計金額", "送料合計", "クーポン利用総額"]
columns_2024_2025_yahoo = ["OrderTime", "Title", "UnitPrice", "TotalMallCouponDiscount", "ShipCharge"]

# --- カラム名を統一 ---
df_2021_2023 = pd.concat(
    [data_2021_2023_rakuten[columns_rakuten], data_2021_2023_yahoo[columns_yahoo]],
    ignore_index=True
).rename(columns={
    "OrderDay": "OrderTime",
    "TotalPrice（商品合計金額）": "TotalPrice",
    "TotalMallCouponDiscount（クーポン値引金額）": "TotalMallCouponDiscount",
    "ShipCharge（送料）": "ShipCharge"
})

# 時刻修正（例: "2021/7/2 8:45" → "2021/7/2 08:45"）
def fix_time_format(s):
    if pd.isna(s):
        return s
    return re.sub(r'(\d{4}/\d{1,2}/\d{1,2}) (\d:\d{2})', r'\1 0\2', str(s))

df_2021_2023['OrderTime'] = df_2021_2023['OrderTime'].apply(fix_time_format)
df_2021_2023['OrderTime'] = pd.to_datetime(df_2021_2023['OrderTime'], errors='coerce')

# 2023-2024 データ
df_2023_2024 = data_2023_2024[columns_2023_2024].copy()
df_2023_2024['OrderTime'] = pd.to_datetime(df_2023_2024['OrderTime'], errors='coerce')

# 2024-2025 楽天データ
df_2024_2025_rakuten = data_2024_2025_rakuten[columns_2024_2025_rakuten].rename(columns={
    "注文日": "OrderTime",
    "商品名": "Title",
    "合計金額": "TotalPrice",
    "送料合計": "ShipCharge",
    "クーポン利用総額": "TotalMallCouponDiscount"
})
df_2024_2025_rakuten['OrderTime'] = pd.to_datetime(df_2024_2025_rakuten['OrderTime'], errors='coerce')

# 2024-2025 Yahooデータ
df_2024_2025_yahoo = data_2024_2025_yahoo[columns_2024_2025_yahoo].rename(columns={
    "UnitPrice": "TotalPrice"
})
df_2024_2025_yahoo['OrderTime'] = pd.to_datetime(df_2024_2025_yahoo['OrderTime'], errors='coerce')

# --- 結合 ---
all_data = pd.concat(
    [df_2021_2023, df_2023_2024, df_2024_2025_rakuten, df_2024_2025_yahoo],
    ignore_index=True
)

# --- 日付を YYYY-MM-DD 表記に変換 ---
all_data['OrderTime'] = all_data['OrderTime'].dt.strftime('%Y-%m-%d')

# --- 昇順ソート & インデックスリセット ---
all_data = all_data.sort_values(by='OrderTime').reset_index(drop=True)

# --- 欠損値は 0 に埋める ---
all_data = all_data.fillna(0)

# --- 確認表示 ---
display(all_data)

# --- 保存（必要なら） ---
all_data.to_csv("../dataset/all_sales_data_2021_2025.csv", index=False, encoding="utf-8-sig")


,OrderTime,Title,TotalPrice,TotalMallCouponDiscount,ShipCharge
0,2021-06-16,美品/東京ソワール D'REREAD ブラックフォーマルワンピース 大きいサイズ 表記15号...,27720,0.0,0
1,2021-06-20,【中古】インディヴィ INDIVI 暖かウール混コート/ジャケット 表記38号(9号/M相当...,20000,0.0,0
2,2021-06-22,【中古】23区 美形パンツスーツ 表記40号(11号/L相当) グレー 薄手 毛ウール混 シ...,18000,0.0,0
3,2021-06-24,【中古】クリスチャンオジャール CHRISTIAN AUJARD 茶ブラウン 7分袖 美柄カ...,7200,0.0,0
4,2021-07-02,【中古】斉藤都世子 toyoko saito ニットの女王 大人ニットワンピース 9号/38...,39500,500.0,0
...,...,...,...,...,...
2558,2025-04-12,美品【中古】バーバリー BURBERRYS スカート 台形スカート 表記 11号 L 40号...,16300,0.0,0
2559,2025-04-12,美品/レリアン Leilian フレアスカート 表記 11号 L 相当 黒 ブラック 毛 ウ...,15400,0.0,0
2560,2025-04-14,美品/ユキトリイ YUKI TORII フォーマルジャケット 9号 M 相当 白 ホワイト ...,28600,0.0,0
2561,2025-04-15,A美品/B3/ビースリー/白/ストレッチ素材/ロゴ釦/茶ステッチライン/お洒落カプリパンツ/...,3800,0.0,600


In [29]:
import pandas as pd, re, unicodedata, pathlib, yaml

# --- 1) 正規化ヘルパー -------------------------------------------------
def normalize(txt: str) -> str:
    if pd.isna(txt):
        return ''
    txt = unicodedata.normalize('NFKC', txt)
    return re.sub(r'\s+', ' ', txt).strip().lower()

# --- 2) キーワード辞書のロード ----------------------------------------
DICT_DIR = pathlib.Path("../dicts")

def load_list(fname):
    return set((DICT_DIR/fname).read_text(encoding='utf-8').strip().splitlines())

KW_AW = load_list("season_autumn_winter.txt")
KW_SS = load_list("season_spring_summer.txt")
KW_SA = load_list("season_spring_autumn.txt")

CATEGORY_KW = yaml.safe_load((DICT_DIR/"category.yaml").read_text())
OVR = yaml.safe_load((DICT_DIR/"exception_overrides.yaml").read_text())
if OVR is None:
    OVR = {}

EXPLICIT = {
    '通年': ['春 夏 秋 冬', 'オールシーズン', '通年', 'シーズンレス'],
    '秋冬': ['秋冬', '秋 冬'],
    '春秋': ['春秋', '春 秋', '秋春', '秋 春'],
    '春夏': ['春夏', '春 夏']
}

# --- 3) 季節ラベル ------------------------------------------------------
def label_season(text):
    t = normalize(text)
    for patt, lbl in OVR.items():
        if patt in t:
            return lbl
    for season, kws in EXPLICIT.items():
        if any(kw in t for kw in kws):
            return season
    if any(k in t for k in KW_AW):
        return '秋冬'
    if any(k in t for k in KW_SS):
        return '春夏'
    if any(k in t for k in KW_SA):
        return '春秋'
    return '通年'

# --- 4) カテゴリラベル --------------------------------------------------
def label_category(text):
    t = normalize(text)
    for cat, kwlist in CATEGORY_KW.items():
        if any(k in t for k in kwlist):
            return cat
    return 'Other'

def extract_brand_candidate(text):
    if pd.isna(text):
        return ''

    text = str(text).strip()

    # ① セールやコンディション語を除去するための正規表現（先頭に複合で現れる場合も想定）
    # 例: "セール★A美品*", "S新品同様【中古】" などを想定
    text = re.sub(
        r'^[☆★◆■＊*/\-\s]*(セール)?[A-Za-zＡ-Ｚａ-ｚ]?[新品未使用同様極超良美中古新同訳上特上やや]*品?[様同相]?[【\[\(]?(中古|used)?[】\]\)]?[/＊*♪☆★◆■\-\s]*',
        '', text, flags=re.IGNORECASE
    ).strip()


    # ② トークン化
    tokens = re.split(r'[＊*/\s♪☆★◆■【】・/・／\-]+', text)
    tokens = [t for t in tokens if t]

    # ③ 除去したいノイズ語
    noise_words = {
        '様', '中古', 'used', '新品', '未使用', '極美品', '美品', '超美品', '良品', '新同',
        'やや美品', '訳あり', '特上美品', 'セール', 'sale'
    }

    # ④ 英字・カナ・漢字を含み、ノイズ語ではないものを上位から2～3語返す
    brand_tokens = []
    for token in tokens:
        if token.lower() not in noise_words and re.search(r'[A-Za-zァ-ヶー一-龥]', token):
            brand_tokens.append(token)
        if len(brand_tokens) >= 3:
            break

    return " ".join(brand_tokens)


# --- 日本語・英語のブランド名を分割 -------------------------------------
def split_japanese_english_brand(text):
    if pd.isna(text):
        return '', ''
    text = str(text).strip()
    tokens = text.split()
    english_tokens = []
    japanese_tokens = []
    for token in tokens:
        if re.fullmatch(r'[A-Za-z0-9\-\']+', token):
            english_tokens.append(token)
        elif re.search(r'[ぁ-んァ-ン一-龥]', token):
            japanese_tokens.append(token)
    return ' '.join(japanese_tokens), ' '.join(english_tokens)


# --- 素材とカテゴリキーワードの読み込み --------------------------------
MATERIAL_KW = yaml.safe_load((DICT_DIR/"material.yaml").read_text())
if MATERIAL_KW is None:
    MATERIAL_KW = {}
MATERIAL_KW_SET = set()
for v in MATERIAL_KW.values():
    MATERIAL_KW_SET.update(v)

CATEGORY_KW_SET = set()
for v in CATEGORY_KW.values():
    CATEGORY_KW_SET.update(v)

# --- ブランド名_日本語 から素材・カテゴリ語を除去して最初のブランドらしさを抽出 ---
def extract_main_japanese_brand(text):
    if pd.isna(text):
        return ''
    
    text = str(text).strip()
    tokens = re.split(r'\s+', text)

    # 「ザ」で始まる場合は後続2単語までセットで返す（例：ザ スーツカンパニー）
    if tokens and tokens[0] == 'ザ':
        return ' '.join(tokens[:3])

    for token in tokens:
        # 素材やカテゴリに部分一致しているものを除外
        if any(mat in token for mat in MATERIAL_KW_SET):
            continue
        if any(cat in token for cat in CATEGORY_KW_SET):
            continue
        if re.fullmatch(r'[a-zA-Z0-9]+', token):  # 半角英数のみは除外
            continue
        if re.search(r'[一-龥ぁ-んァ-ヶーＡ-Ｚａ-ｚ０-９]', token):
            return token

    return ''

# --- 6) ラベリング処理 --------------------------------------------------
if __name__ == "__main__":
    df = pd.read_csv("../dataset/all_sales_data_2021_2025.csv", encoding="utf-8-sig")
    df['title_norm']     = df['Title'].apply(normalize)
    df['季節ラベル']       = df['title_norm'].apply(label_season)
    df['カテゴリラベル']     = df['title_norm'].apply(label_category)
    df['ブランド名候補']     = df['Title'].apply(extract_brand_candidate)
    df[['ブランド名_日本語', 'ブランド名_英語']] = df['ブランド名候補'].apply(lambda x: pd.Series(split_japanese_english_brand(x)))
    df['ブランド名_日本語_clean'] = df['ブランド名_日本語'].apply(extract_main_japanese_brand)

    # 保存ファイル名
    out = "../dataset/all_sales_data_2021_2025_labeled_draft.csv"
    df.to_csv(out, index=False, encoding="utf-8-sig")

    print("Finished labeling. Saved to:", out)



Finished labeling. Saved to: ../dataset/all_sales_data_2021_2025_labeled.csv


In [71]:
import pandas as pd
import jpholiday

# --- 1. CSV 読み込みと基本整形 ---
df = pd.read_csv("../dataset/all_sales_data_2021-2025.csv", encoding="utf-8-sig")
df_weather = pd.read_csv("../csv/Related/weather_with_sales_split.csv", encoding="utf-8-sig")
df = df.drop(columns=["Title"])

df['OrderTime'] = pd.to_datetime(df['OrderTime'])
df["Week"] = df["OrderTime"].dt.day_name()
df["Holiday"] = df["OrderTime"].apply(lambda x: 1 if jpholiday.is_holiday(x) else 0)

df['TotalMallCouponDiscount'] = df['TotalMallCouponDiscount'].abs()
df['ShipCharge'] = df['ShipCharge'].abs()

# --- 2. ブランドごとの平均売上金額の算出 ---
brand_avg = df.groupby("Brand_English")["TotalPrice"].mean().rename("brand_avg_sales")
df = df.merge(brand_avg, on="Brand_English", how="left")

# --- 3. 価格帯カテゴリラベルの定義 ---
def label_price_tier(avg):
    if avg < 1000:
        return "Very Low"
    elif avg < 5000:
        return "Low"
    elif avg < 10000:
        return "Medium"
    elif avg < 15000:
        return "High"
    elif avg < 20000:
        return "Very High"
    else:
        return "Extremely High"

df["price_tier"] = df["brand_avg_sales"].apply(label_price_tier)

# --- 4. One-Hot Encoding ---
df = pd.concat([df, pd.get_dummies(df["price_tier"], prefix="Tier", dtype=int)], axis=1)

# --- 5. One-Hot Encoding for Season, Category, Week ---
df = df[df["Season"] != "通年"].reset_index(drop=True)
df = pd.get_dummies(df, columns=["Season", "Category", "Week"], prefix=["Season", "Category", "Week"], dtype=int)

# --- 6. Holiday を 0/1 に変換（True/False → 1/0）---
if df["Holiday"].dtype == bool:
    df["Holiday"] = df["Holiday"].astype(int)

# --- 7. 不要列の削除 ---
df = df.drop(columns=["Brand_English", "Brand_Japanese", "price_tier", "brand_avg_sales"])

# --- 8. 日別集計 (数値のみ対象) ---
df_daily = df.groupby("OrderTime").sum().reset_index()

# --- 9. 日付連続埋め ---
all_days = pd.DataFrame(pd.date_range(start="2021-06-16", end="2025-04-18", freq="D"), columns=["OrderTime"])
df_daily = all_days.merge(df_daily, on="OrderTime", how="left")
df_daily.fillna(0, inplace=True)

# --- 10. 気象＋セールデータと結合 ---
df_weather["date"] = pd.to_datetime(df_weather["date"])
df_final = df_daily.merge(df_weather, left_on="OrderTime", right_on="date", how="left").drop(columns=["date"])

# --- 11. 保存 ---
df_final.to_csv("../dataset/daily_sales_encoded_weather_merged.csv", index=False, encoding="utf-8-sig")
print("✅ 日別の売上データに気象＋セール情報を結合し、保存しました。")


✅ 日別の売上データに気象＋セール情報を結合し、保存しました。


In [85]:
import pandas as pd
import re

# -------------------------------------------------
# 0) 共通関数 ―― 文字列を7大カテゴリへまとめる
# -------------------------------------------------
def classify_weather(text: str) -> str:
    """
    詳細な天気概況文字列 → 主要 7 クラスへマッピング
      晴 / 曇 / 雨 / 雪 / 雷 / 複合(晴雨混在) / その他
    """
    if pd.isna(text):
        return "その他"
    t = re.sub(r"\s+", "", text)  # 空白・改行除去

    # ① 雷（最優先）
    if '雷' in t:
        return '雷'

    # ② 雪・みぞれ
    if any(k in t for k in ['雪', 'みぞれ']):
        return '雪'

    # ③ 雨
    if '雨' in t or '霧雨' in t:
        # 雨 + 晴/曇/薄曇 が混在 → “複合”
        if any(k in t for k in ['晴', '曇', '薄曇']):
            return '複合(晴雨混在)'
        return '雨'

    # ④ 晴
    if any(k in t for k in ['快晴', '晴', '薄曇']):
        return '晴'

    # ⑤ 曇
    if '曇' in t:
        return '曇'

    # ⑥ その他（霧など）
    return 'その他'


# -------------------------------------------------
# 1) データ読み込み
# -------------------------------------------------
df = pd.read_csv("../dataset/daily_sales_encoded_weather_merged.csv",
                 encoding="utf-8-sig")

# 使わない列はここで削除しておく
df = df.drop(columns=["rakuten_sale_name", "yahoo_sale_name"])

# -------------------------------------------------
# 2) 昼・夜それぞれを 7 クラスに変換
# -------------------------------------------------
day_col  = "天気概況(昼：06時～18時)"
night_col = "天気概況(夜：18時～翌日06時)"

df["weather_day_cat"]   = df[day_col].apply(classify_weather)
df["weather_night_cat"] = df[night_col].apply(classify_weather)

# -------------------------------------------------
# 3) One-Hot Encoding
#     ・昼：prefix=weather_day_
#     ・夜：prefix=weather_night_
# -------------------------------------------------
day_ohe   = pd.get_dummies(df["weather_day_cat"],
                           prefix="weather_day", dtype=int)
night_ohe = pd.get_dummies(df["weather_night_cat"],
                           prefix="weather_night", dtype=int)

df = pd.concat([df, day_ohe, night_ohe], axis=1)

# -------------------------------------------------
# 4) 結果確認
# -------------------------------------------------
pd.set_option("display.max_rows", None)
print("▼ 昼カテゴリ件数\n",   df["weather_day_cat"].value_counts())
print("\n▼ 夜カテゴリ件数\n", df["weather_night_cat"].value_counts())

# -------------------------------------------------
# 5) 保存
# -------------------------------------------------
out_path = "../dataset/daily_sales_weather_encoded.csv"
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"✅ 変換済みデータを保存しました: {out_path}")


▼ 昼カテゴリ件数
 weather_day_cat
晴           879
複合(晴雨混在)    328
曇           109
雨            44
雷            33
雪            10
Name: count, dtype: int64

▼ 夜カテゴリ件数
 weather_night_cat
晴           860
複合(晴雨混在)    345
曇           111
雨            40
雷            36
雪            11
Name: count, dtype: int64
✅ 変換済みデータを保存しました: ../dataset/daily_sales_weather_encoded.csv


In [63]:
import pandas as pd

# --- ファイルパスの設定 ---
weather_path = "../csv/Related/weather_data_2021-2025.csv"
sale_path = "../csv/Related/sale_info.csv"

# --- 1. 気象データの開始行特定 ---
with open(weather_path, encoding="utf-8-sig") as f:
    lines = f.readlines()

start_idx = None
for i, line in enumerate(lines):
    if "年月日" in line:
        start_idx = i
        break

# --- 2. 気象データ読み込み ---
weather_df = pd.read_csv(weather_path, encoding="utf-8-sig", skiprows=start_idx)
weather_df.rename(columns={weather_df.columns[0]: "date"}, inplace=True)
weather_df["date"] = pd.to_datetime(weather_df["date"], errors="coerce")

# --- 3. セールデータ読み込み ---
sale_df = pd.read_csv(sale_path, encoding="utf-8")
sale_df["date"] = pd.to_datetime(sale_df["date"], errors="coerce")

# --- 4. プラットフォーム別に辞書化してマッピング ---
rakuten_dict = sale_df[sale_df["platform"] == "楽天市場"].set_index("date")["sale_name"].to_dict()
yahoo_dict = sale_df[sale_df["platform"].str.contains("Yahoo")].set_index("date")["sale_name"].to_dict()

# --- 5. セール名のマッピング ---
weather_df["rakuten_sale_name"] = weather_df["date"].map(rakuten_dict)
weather_df["yahoo_sale_name"] = weather_df["date"].map(yahoo_dict)

# --- 6. フラグの付与（セールがあるかどうか） ---
weather_df["rakuten_sale_flag"] = weather_df["rakuten_sale_name"].notnull().astype(int)
weather_df["yahoo_sale_flag"] = weather_df["yahoo_sale_name"].notnull().astype(int)

# --- 7. 保存 ---
output_path = "../csv/Related/weather_with_sales_split.csv"
weather_df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"✅ セール情報付き気象データを保存しました: {output_path}")


✅ セール情報付き気象データを保存しました: ../csv/Related/weather_with_sales_split.csv


In [86]:
import pandas as pd

# ---------- 1. Load ----------
df = pd.read_csv(
    "../dataset/daily_sales_weather_encoded.csv", encoding="utf-8-sig"
)

# ---------- 2. Rename map ----------
rename_map = {
    # もともと日本語の列 -----------------------------
    "OrderTime": "order_time",                       # すでに英語だけど小文字へ
    "TotalPrice": "total_price",
    "TotalMallCouponDiscount": "mall_coupon_discount",
    "ShipCharge": "ship_charge",
    "Holiday": "jp_holiday",

    "平均気温(℃)": "temp_avg_c",
    "最高気温(℃)": "temp_max_c",
    "最低気温(℃)": "temp_min_c",
    "降水量の合計(mm)": "precip_mm",
    "日照時間(時間)": "sunshine_h",

    "weather_night_cat": "weather_night_cat",

    # One-hot 元の列名（prefix 部分だけ英語化しておけば OK）
    "Tier_Extremely High": "tier_extremely_high",
    "Tier_High": "tier_high",
    "Tier_Low": "tier_low",
    "Tier_Medium": "tier_medium",
    "Tier_Very High": "tier_very_high",
    "Tier_Very Low": "tier_very_low",

    "Season_春夏": "season_spring_summer",
    "Season_春秋": "season_spring_autumn",
    "Season_秋冬": "season_autumn_winter",

    "Category_Accessory": "cat_accessory",
    "Category_Bottoms": "cat_bottoms",
    "Category_Dress": "cat_dress",
    "Category_Other": "cat_other",
    "Category_Outer": "cat_outer",
    "Category_Suit": "cat_suit",
    "Category_Tops": "cat_tops",

    "Week_Friday": "week_friday",
    "Week_Monday": "week_monday",
    "Week_Saturday": "week_saturday",
    "Week_Sunday": "week_sunday",
    "Week_Thursday": "week_thursday",
    "Week_Tuesday": "week_tuesday",
    "Week_Wednesday": "week_wednesday",

    "SaleEventFlag_rakuten": "sale_flag_rakuten",
    "SaleEventFlag_yahoo": "sale_flag_yahoo",

    # weather day/night one-hot
    "weather_day_晴": "wday_clear",
    "weather_day_曇": "wday_cloudy",
    "weather_day_複合(晴雨混在)": "wday_mix",
    "weather_day_雨": "wday_rain",
    "weather_day_雪": "wday_snow",
    "weather_day_雷": "wday_thunder",

    "weather_night_晴": "wnight_clear",
    "weather_night_曇": "wnight_cloudy",
    "weather_night_複合(晴雨混在)": "wnight_mix",
    "weather_night_雨": "wnight_rain",
    "weather_night_雪": "wnight_snow",
    "weather_night_雷": "wnight_thunder",
}

# ---------- 3. Apply ----------
df = df.rename(columns=rename_map)

# ---------- 4. Save ----------
out_path = "../dataset/daily_usedcloting_sales_data.csv"
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"✅ Saved with English column names: {out_path}")


✅ Saved with English column names: ../dataset/daily_usedcloting_sales_data.csv


In [87]:
import pandas as pd

# ---------- 1. Load ----------
src_path  = "../dataset/forecast_dedicated/daily_usedcloting_sales_data.csv"
save_path = "../dataset/forecast_dedicated/daily_usedcloting_sales_data_filled.csv"

try:
    df = pd.read_csv(src_path, encoding="utf-8-sig")
except UnicodeDecodeError:
    df = pd.read_csv(src_path, encoding="cp932")

# ---------- 2. sunshine_h ― 月毎中央値で欠損補完 ----------
df["order_time"] = pd.to_datetime(df["order_time"], errors="coerce")
df["month"]      = df["order_time"].dt.month       # 一時列

df["sunshine_h"] = (
    df.groupby("month")["sunshine_h"]
      .transform(lambda s: s.fillna(s.median()))
)

df = df.drop(columns="month")                      # 後片付け

# ---------- 3. Missing-value report ----------
na_cnt   = df.isna().sum()
na_ratio = (na_cnt / len(df)).round(4)

missing_report = (
    pd.DataFrame({"missing_cnt": na_cnt, "missing_ratio": na_ratio})
      .sort_values("missing_cnt", ascending=False)
)

print("▼ Missing-value report (top 30)")
print(missing_report.head(30))
print("\n✅ Any missing values?  -->", df.isna().values.any())

# ---------- 4. Save ----------
df.to_csv(save_path, index=False, encoding="utf-8-sig")
print(f"✅ Filled dataset saved to: {save_path}")


▼ Missing-value report (top 30)
                      missing_cnt  missing_ratio
order_time                      0            0.0
week_thursday                   0            0.0
week_wednesday                  0            0.0
temp_avg_c                      0            0.0
temp_max_c                      0            0.0
temp_min_c                      0            0.0
precip_mm                       0            0.0
sunshine_h                      0            0.0
sale_flag_rakuten               0            0.0
sale_flag_yahoo                 0            0.0
weather_day_cat                 0            0.0
weather_night_cat               0            0.0
wday_clear                      0            0.0
wday_cloudy                     0            0.0
wday_mix                        0            0.0
wday_rain                       0            0.0
wday_snow                       0            0.0
wday_thunder                    0            0.0
wnight_clear                    0    

In [88]:
import pandas as pd

# ▼ 1) 読み込み  ----------------------------------------------------------
PATH_IN  = "../dataset/forecast_dedicated/daily_usedcloting_sales_data_filled.csv"
PATH_OUT = "../dataset/forecast_dedicated/daily_usedcloting_sales_data_filled_weekfix.csv"

df = pd.read_csv(PATH_IN, encoding="utf-8-sig")

# ▼ 2) 既存の曜日ダミー列を一旦削除（残っている場合のみ） -------------
week_cols = [c for c in df.columns if c.startswith("week_")]
df = df.drop(columns=week_cols, errors="ignore")

# ▼ 3) order_time → 曜日へ変換し One-Hot Encoding -----------------------
df["order_time"] = pd.to_datetime(df["order_time"])
df["weekday_str"] = df["order_time"].dt.day_name()          # e.g. "Monday"

df = pd.get_dummies(
        df,
        columns=["weekday_str"],
        prefix="week",
        dtype=int
     )

# ▼ 4) 保存 --------------------------------------------------------------
df.to_csv(PATH_OUT, index=False, encoding="utf-8-sig")
print(f"✅ 曜日ダミーを再生成して保存しました: {PATH_OUT}")


✅ 曜日ダミーを再生成して保存しました: ../dataset/forecast_dedicated/daily_usedcloting_sales_data_filled_weekfix.csv
